# Assignment 3: All Pairs Similarity

## Setup & Dataset Retrieval

In [1]:
import os.path
import pickle
import random
import pandas as pd
from tqdm import tqdm

import src.utilities.io as io
import src.utilities.preprocess as pp
import src.apds.docsim_sequential as ds_seq
import src.apds.docsim_mapreduce_spark as ds_mr

/Users/a/miniforge3/envs/sbruch-assignment-Lwmd/lib/python3.10/site-packages/beir/util.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package punkt to /Users/a/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/a/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/a/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
documents, data_path = io.get_data() #trec-covid as default

2023-05-31 00:30:55 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2023-05-31 00:30:57 - Loaded 171332 TEST Documents.
2023-05-31 00:30:57 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

filter out the documents with less than 10 words.

In [3]:
documents = {doc_id: doc for doc_id, doc in documents.items() if len(doc['text'].split()) >= 10 and doc['text'].strip() != ''}

In [4]:
for doc_id, doc in list(documents.items())[:10]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ug7v899j: 262 words
Document 02tnwd4m: 142 words
Document ejv2xln0: 219 words
Document 2b73a28n: 68 words
Document 9785vg6d: 110 words
Document zjufx4fo: 174 words
Document 5yhe786e: 113 words
Document 8zchiykl: 83 words
Document 8qnrcgnk: 188 words
Document jg13scgo: 189 words


### Sort the documents by their length (Longest to Shortest)

In [5]:
sort_docs = sorted(documents.items(), key=lambda x: len(x[1]['text']), reverse=True)
sorted_docs: pp.Documents = {}
for doc_id, doc in sort_docs:
    sorted_docs[doc_id] = doc

In [6]:
len(sorted_docs)

128592

In [7]:
for doc_id, doc in list(sorted_docs.items())[:50]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document ij3ncdb6: 18000 words
Document c4pt07zk: 15448 words
Document 1vimqhdp: 12745 words
Document pd1g119c: 4642 words
Document gvh0wdxn: 3507 words
Document jkrj0lbm: 3783 words
Document jkvr9lq2: 3679 words
Document y00pz4fd: 1705 words
Document lqdgvsaq: 1627 words
Document vslevssr: 1504 words
Document o1vxgzpj: 1437 words
Document bt59txxp: 1474 words
Document yt9rj42s: 1460 words
Document qapwb5e4: 1529 words
Document rff3ikak: 1490 words
Document 0prx11md: 1310 words
Document 84hh0mg9: 1312 words
Document uumzl37q: 1053 words
Document vr7vm64u: 1097 words
Document v45cb2xq: 1136 words
Document yn7pu9i8: 1204 words
Document t473tbm9: 1271 words
Document nq0qe8qf: 1151 words
Document 16b8drw2: 1163 words
Document 4cbjbo7t: 1236 words
Document tvs1snq8: 1139 words
Document 2wb007gf: 1102 words
Document st1epx6u: 1114 words
Document syl7wq15: 1114 words
Document dt3li3bk: 1054 words
Document ec798c6s: 1057 words
Document xvnxkcxq: 1032 words
Document rxvxf34z: 1039 words
Documen

In [8]:
for doc_id, doc in list(sorted_docs.items())[-50:]:
    num_words = len(doc["text"].split())
    print(f"Document {doc_id}: {num_words} words")

Document denmp8yh: 10 words
Document 1k5h0tqi: 10 words
Document 46suh53t: 10 words
Document rmlklqwo: 15 words
Document vetdsk48: 10 words
Document h29uwe1d: 11 words
Document e4t19dza: 14 words
Document kg4oljan: 10 words
Document pfxyz3q3: 10 words
Document dvlwbho2: 11 words
Document fj9le8dx: 10 words
Document bg1amnoh: 11 words
Document 60p2a1i2: 10 words
Document suqvyeac: 13 words
Document h6r6qdna: 11 words
Document 6xqmlgr2: 10 words
Document igw288oj: 15 words
Document wt8ucsjr: 10 words
Document cjw88eoq: 11 words
Document n2v9kpm2: 12 words
Document srrdh09m: 12 words
Document l9n7mklv: 11 words
Document ob9i9t04: 11 words
Document 8j184di4: 11 words
Document yfs4kbus: 10 words
Document 5rmvdtrb: 12 words
Document kxbjtyjv: 11 words
Document pcl5bn54: 12 words
Document 08grs6m7: 10 words
Document sqos28f3: 12 words
Document isn31iqt: 12 words
Document 944w4qgq: 11 words
Document 1r60beaa: 10 words
Document 3nxp0j41: 10 words
Document i9aieai9: 11 words
Document i6b2bg6a: 1

This dictionary is used to map the index of a document to the corresponding text_id in the original dataset.
This index-to-ID mappings is useful for keeping track of the original text_id of the documents.

In [9]:
doc_idx_to_id = {i: doc_id for i, doc_id in enumerate(sorted_docs.keys())}

## Due to the size, we will use different dataset samples
(1000 longest docs, 1000 shortest docs, 1000 random docs)

### Define the paths to save and read the samples and their mappings

In [10]:
sample_size = 400
samples_dir = os.path.join(data_path, "samples_" + str(sample_size))
if not os.path.exists(samples_dir):
    os.mkdir(samples_dir)

random_docs_idx_path = os.path.join(samples_dir, "random_docs_idx.pkl")

In [11]:
if not os.path.exists(random_docs_idx_path):
    random_docs_idx = random.sample(range(len(sorted_docs)), k=sample_size)

    with open(random_docs_idx_path, "wb") as f:
        pickle.dump(random_docs_idx, f)
else:
    with open(random_docs_idx_path, "rb") as f:
        random_docs_idx = pickle.load(f)

longest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[:sample_size]:
    longest_docs[doc_id] = doc
longest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(sample_size))}

shortest_docs = pp.Documents = {}
for doc_id, doc in list(sorted_docs.items())[-sample_size:]:
    shortest_docs[doc_id] = doc
shortest_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(range(len(sorted_docs)-sample_size, len(sorted_docs)))}

random_docs = pp.Documents = {}
s_docs = list(sorted_docs.items())
for index in random_docs_idx:
    doc_id, doc = s_docs[index]
    random_docs[doc_id] = doc
random_doc_idx_to_id = {i:doc_idx_to_id[idx] for i, idx in enumerate(random_docs_idx)}

Mapping is useful if we want to retrieve the original documents corresponding to the randomly chosen documents in random_docs

In [12]:
samples = [("longest_docs", longest_docs, longest_doc_idx_to_id),
           ("shortest_docs", shortest_docs, shortest_doc_idx_to_id),
           ("random_docs", random_docs, random_doc_idx_to_id)]

## Tokenization
Document cleaning and tokenization

In [13]:
tokens_dir = os.path.join(samples_dir, "tokens")
if not os.path.exists(tokens_dir):
    os.mkdir(tokens_dir)

docs_tokens_path = os.path.join(tokens_dir, "doc_tokens.pkl")

In [14]:
if not os.path.exists(docs_tokens_path):
    tokenized_samples = []
    for name, docs, idx_to_id in samples:
        tokenized_docs: pp.TokenizedDocuments = pp.get_tokenized_documents(docs)
        tokenized_samples.append((name, tokenized_docs, idx_to_id))

    with open(docs_tokens_path, "wb") as f:
        pickle.dump(tokenized_samples, f)
else:
    with open(docs_tokens_path, "rb") as f:
        tokenized_samples = pickle.load(f)

Tokenizing documents: 100%|██████████| 400/400 [00:00<00:00, 489.30it/s]


## VECTORIZE

In [15]:
tfidf_dir = os.path.join(samples_dir, "tfidf")
if not os.path.exists(tfidf_dir):
    os.mkdir(tfidf_dir)

tfidf_results_path = os.path.join(tfidf_dir, "tfidf_docs.pkl")

In [16]:
if not os.path.exists(tfidf_results_path):
    tfidf_docs = []
    for name, docs, idx_to_id in tqdm(tokenized_samples):
        vectorized_docs, time_taken = pp.vectorize(docs)
        tfidf_docs.append((name, vectorized_docs, time_taken, idx_to_id))

    with open(tfidf_results_path, "wb") as f:
        pickle.dump(tfidf_docs, f)
else:
    with open(tfidf_results_path, "rb") as f:
        tfidf_docs = pickle.load(f)

100%|██████████| 3/3 [00:00<00:00,  8.48it/s]


In [17]:
for name, _, time_taken, _ in tfidf_docs:
    print(f"Time taken to vectorize {name}: {time_taken:.6f} seconds")

Time taken to vectorize longest_docs: 0.264694 seconds
Time taken to vectorize shortest_docs: 0.010091 seconds
Time taken to vectorize random_docs: 0.066280 seconds


## Setup

In [18]:
thresholds = [0.3, 0.5, 0.7, 0.9] # Chosen thresholds

## Sequential APDS

In [19]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path = os.path.join(seq_results_dir, "seq_results.pkl")

In [20]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path,
                                       samples_dir=samples_dir,
                                       heuristic=False)

# if not os.path.exists(seq_results_path):
#     for name, vectorized_docs, tfidf_time, idx_to_id in tqdm(tfidf_docs):
#         seq_results = []
#         for threshold in thresholds:
#             similar_pairs, information = ds_seq.sequential_apds(tfidf_matrix=vectorized_docs, sample_name=name, threshold=threshold, tfidf_time=tfidf_time)
#             mapped_pairs = ds_seq.map_doc_idx_to_id(similar_pairs, idx_to_id)
#             seq_results.append((similar_pairs, mapped_pairs, information))
#             ds_seq.save_seq_result_csv(all_pairs=mapped_pairs, ds_name=name, threshold=threshold, samples_dir=samples_dir)
#         all_seq_results[name] = seq_results
#
#     with open(seq_results_path, "wb") as f:
#         pickle.dump(all_seq_results, f)
# else:
#     with open(seq_results_path, "rb") as f:
#         all_seq_results = pickle.load(f)

100%|██████████| 3/3 [00:00<00:00,  8.31it/s]


### longest 1000

In [21]:
ds_seq.print_results(all_seq_results, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 721
tfidf_time: 0.2646939754486084
cosine_time: 0.027189016342163086
find_time: 0.016483068466186523
total_time: 0.308366060256958

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 137
tfidf_time: 0.2646939754486084
cosine_time: 0.02305912971496582
find_time: 0.016518831253051758
total_time: 0.304271936416626

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 107
tfidf_time: 0.2646939754486084
cosine_time: 0.023602008819580078
find_time: 0.01772785186767578
total_time: 0.30602383613586426

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 85
tfidf_time: 0.2646939754486084
cosine_time: 0.03056478500366211
find_time: 0.016540050506591797
total_time: 0.3117988109588623



In [22]:
ds_seq.print_first_pair(all_seq_results, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5275 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [23]:
ds_seq.print_results(all_seq_results, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 319
tfidf_time: 0.010090827941894531
cosine_time: 0.0019750595092773438
find_time: 0.015899181365966797
total_time: 0.027965068817138672

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 94
tfidf_time: 0.010090827941894531
cosine_time: 0.001622915267944336
find_time: 0.020471811294555664
total_time: 0.03218555450439453

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 53
tfidf_time: 0.010090827941894531
cosine_time: 0.0016219615936279297
find_time: 0.019790172576904297
total_time: 0.03150296211242676

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 51
tfidf_time: 0.010090827941894531
cosine_time: 0.0016357898712158203
find_time: 0.020119905471801758
total_time: 0.03184652328491211



In [24]:
ds_seq.print_first_pair(all_seq_results, "shortest_docs", tokenized_samples)

8qx7s4k6 and 0ioxeuya have 0.4062 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

0ioxeuya:
caring orthopaedic patient affected elective surgery suspension temporary suspension elective surgery affect thousand patient currently awaiting orthopaedic surgery

8qx7s4k6 and gbrgk5hr have 0.8992 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

gbrgk5hr:
collateral damage covid-19 pandemic nutritional quality physical activity perspective south korea novel coronavirus sars-cov-2 emerged city wuhan china december 2019 spread worldwide

8qx7s4k6 and gbrgk5hr have 0.8992 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

gbrgk5hr:
collateral damage covid-19 pandemic nutritional quality physical activity perspective south korea novel coronavirus sars-cov-

### random 1000

In [25]:
ds_seq.print_results(all_seq_results, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 36
tfidf_time: 0.06627988815307617
cosine_time: 0.007061004638671875
find_time: 0.01522207260131836
total_time: 0.0885629653930664

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 1
tfidf_time: 0.06627988815307617
cosine_time: 0.006779909133911133
find_time: 0.015395879745483398
total_time: 0.0884556770324707

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 0
tfidf_time: 0.06627988815307617
cosine_time: 0.007441997528076172
find_time: 0.018231868743896484
total_time: 0.09195375442504883

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 0
tfidf_time: 0.06627988815307617
cosine_time: 0.006910085678100586
find_time: 0.015208005905151367
total_time: 0.08839797973632812



In [26]:
ds_seq.print_first_pair(all_seq_results, "random_docs", tokenized_samples)

u3ir9h8k and fqvrc9rh have 0.3201 similarity

u3ir9h8k:
mesenchymal stem cell therapy acute respiratory distress syndrome basic clinic 2019 novel coronavirus disease covid-19 caused severe acute respiratory syndrome coronavirus 2 sars-cov-2 occurred china around world sars-cov-2-infected patient severe pneumonia rapidly develop acute respiratory distress syndrome ards die multiple organ failure despite advance supportive care approach ards still associated high mortality morbidity mesenchymal stem cell msc -based therapy may potential alternative strategy treating ards targeting various pathophysiological event ards releasing variety paracrine factor extracellular vesicle msc exert anti-inflammatory anti-apoptotic anti-microbial pro-angiogenic effect promote bacterial alveolar fluid clearance disrupt pulmonary endothelial epithelial cell damage eventually avoiding lung distal organ injury rescue patient ards increasing number experimental animal study early clinical study verify safety

## Sequential APDS with Heuristic

In [27]:
seq_results_dir = os.path.join(samples_dir, "seq_result")
if not os.path.exists(seq_results_dir):
    os.mkdir(seq_results_dir)

seq_results_path_h = os.path.join(seq_results_dir, "seq_results_h.pkl")

In [28]:
#sample: pairs indices, pairs docs id, additional information
all_seq_results_h = ds_seq.exec_seq_apds(tfidf_docs=tfidf_docs,
                                       thresholds=thresholds,
                                       seq_results_path=seq_results_path_h,
                                       samples_dir=samples_dir,
                                       heuristic=True)

100%|██████████| 3/3 [00:01<00:00,  2.67it/s]


### longest 1000

In [29]:
ds_seq.print_results(all_seq_results_h, "longest_docs")

--Run info--
sample_name: longest_docs
threshold: 0.3
pairs_count: 721
tfidf_time: 0.2646939754486084
cosine_time: 0.028143882751464844
find_time: 0.15317296981811523
total_time: 0.4460108280181885

--Run info--
sample_name: longest_docs
threshold: 0.5
pairs_count: 137
tfidf_time: 0.2646939754486084
cosine_time: 0.02734088897705078
find_time: 0.08175516128540039
total_time: 0.37379002571105957

--Run info--
sample_name: longest_docs
threshold: 0.7
pairs_count: 107
tfidf_time: 0.2646939754486084
cosine_time: 0.03130483627319336
find_time: 0.07268595695495605
total_time: 0.3686847686767578

--Run info--
sample_name: longest_docs
threshold: 0.9
pairs_count: 85
tfidf_time: 0.2646939754486084
cosine_time: 0.02866387367248535
find_time: 0.08469486236572266
total_time: 0.3780527114868164



In [30]:
ds_seq.print_first_pair(all_seq_results_h, "longest_docs", tokenized_samples)

1vimqhdp and pd1g119c have 0.5275 similarity

1vimqhdp:
36th international symposium intensive care emergency medicine brussels belgium 15-18 march 2016 p001 sepsis impairs capillary response within hypoxic capillary decrease erythrocyte oxygen-dependent atp efflux r. m. bateman m. d. sharpe j. e. jagger c. g. elli p002 lower serum immunoglobulin g2 level predispose severe flu j. solé-violán m. lópez-rodríguez e. herrera-ramos j. ruíz-hernández l. borderías j. horcajada n. gonzález-quevedo o. raja m. briones f. rodríguez de castro c. rodríguez gallego p003 brain protective effect intravenous immunoglobulin inhibition complement activation apoptosis rat model sepsis f. esen g. orhun p. ergin ozcan e. senturk c. ugur yilmaz n. orhan n. arican m. kaya m. kucukerden m. giris u. akcan s. bilgic gazioglu e. tuzun p004 adenosine a1 receptor dysfunction associated leukopenia possible mechanism sepsis-induced leukopenia r. riff o. naamani a. douvdevani p005 analysis neutrophil hyper spectral im

### shortest 1000

In [31]:
ds_seq.print_results(all_seq_results_h, "shortest_docs")

--Run info--
sample_name: shortest_docs
threshold: 0.3
pairs_count: 319
tfidf_time: 0.010090827941894531
cosine_time: 0.002579927444458008
find_time: 0.06151604652404785
total_time: 0.07418680191040039

--Run info--
sample_name: shortest_docs
threshold: 0.5
pairs_count: 94
tfidf_time: 0.010090827941894531
cosine_time: 0.0015537738800048828
find_time: 0.06222224235534668
total_time: 0.0738668441772461

--Run info--
sample_name: shortest_docs
threshold: 0.7
pairs_count: 53
tfidf_time: 0.010090827941894531
cosine_time: 0.0019168853759765625
find_time: 0.06621432304382324
total_time: 0.07822203636169434

--Run info--
sample_name: shortest_docs
threshold: 0.9
pairs_count: 51
tfidf_time: 0.010090827941894531
cosine_time: 0.0015659332275390625
find_time: 0.05921006202697754
total_time: 0.07086682319641113



In [32]:
ds_seq.print_first_pair(all_seq_results_h, "shortest_docs", tokenized_samples)

8qx7s4k6 and 0ioxeuya have 0.4062 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

0ioxeuya:
caring orthopaedic patient affected elective surgery suspension temporary suspension elective surgery affect thousand patient currently awaiting orthopaedic surgery

8qx7s4k6 and gbrgk5hr have 0.8992 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

gbrgk5hr:
collateral damage covid-19 pandemic nutritional quality physical activity perspective south korea novel coronavirus sars-cov-2 emerged city wuhan china december 2019 spread worldwide

8qx7s4k6 and gbrgk5hr have 0.8992 similarity

8qx7s4k6:
graduating lockdown fabian river recent graduate rep bva council spare thought graduating covid-19 pandemic

gbrgk5hr:
collateral damage covid-19 pandemic nutritional quality physical activity perspective south korea novel coronavirus sars-cov-

### random 1000

In [33]:
ds_seq.print_results(all_seq_results_h, "random_docs")

--Run info--
sample_name: random_docs
threshold: 0.3
pairs_count: 36
tfidf_time: 0.06627988815307617
cosine_time: 0.009690999984741211
find_time: 0.06356406211853027
total_time: 0.13953495025634766

--Run info--
sample_name: random_docs
threshold: 0.5
pairs_count: 1
tfidf_time: 0.06627988815307617
cosine_time: 0.007269859313964844
find_time: 0.06570792198181152
total_time: 0.13925766944885254

--Run info--
sample_name: random_docs
threshold: 0.7
pairs_count: 0
tfidf_time: 0.06627988815307617
cosine_time: 0.0067598819732666016
find_time: 0.08321690559387207
total_time: 0.15625667572021484

--Run info--
sample_name: random_docs
threshold: 0.9
pairs_count: 0
tfidf_time: 0.06627988815307617
cosine_time: 0.006684064865112305
find_time: 0.06174325942993164
total_time: 0.13470721244812012



In [34]:
ds_seq.print_first_pair(all_seq_results_h, "random_docs", tokenized_samples)

u3ir9h8k and fqvrc9rh have 0.3201 similarity

u3ir9h8k:
mesenchymal stem cell therapy acute respiratory distress syndrome basic clinic 2019 novel coronavirus disease covid-19 caused severe acute respiratory syndrome coronavirus 2 sars-cov-2 occurred china around world sars-cov-2-infected patient severe pneumonia rapidly develop acute respiratory distress syndrome ards die multiple organ failure despite advance supportive care approach ards still associated high mortality morbidity mesenchymal stem cell msc -based therapy may potential alternative strategy treating ards targeting various pathophysiological event ards releasing variety paracrine factor extracellular vesicle msc exert anti-inflammatory anti-apoptotic anti-microbial pro-angiogenic effect promote bacterial alveolar fluid clearance disrupt pulmonary endothelial epithelial cell damage eventually avoiding lung distal organ injury rescue patient ards increasing number experimental animal study early clinical study verify safety

## Setup

In [35]:
thresholds = [0.3, 0.5, 0.7, 0.9] # Choosen thresholds
n_slices = [4, 7, 10] # Choosen numslices factors N_PARTITIONS
n_executors = [4, 7, 10] # n_executors
# thresholds =[0.3]
# n_slices = [4]
# n_executors = [4]

## Parallel APDS

In [36]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path = os.path.join(mr_results_dir, "mr_results.pkl")

In [37]:
# for collection_name, collection_docs, _ in samples:
all_mr_results = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path,
                                    samples_dir=samples_dir,
                                    heuristic=False)

# if not os.path.exists(mr_results_path):
#     for name, vectorized_docs, tfidf_time, idx_to_id in tqdm(tfidf_docs):
#         mr_results_th = {}
#         for threshold in thresholds:
#             mr_results = []
#             for workers in n_executors:
#                 for slices in n_slices:
#                     print(f'\n----working on {name}, threshold: {threshold}----')
#                     print(f'\n----workers: {workers}, slices: {slices}----')
#
#                     doc_pairs, doc_pairs_id, doc_pairs_info = ds_mr.spark_apds(ds_name=name,
#                                                                                tfidf_mat=vectorized_docs,
#                                                                                idx_to_id = idx_to_id,
#                                                                                threshold=threshold,
#                                                                                tfidf_time=tfidf_time,
#                                                                                n_executors=workers,
#                                                                                n_slices=slices,
#                                                                                heuristic=False)
#                     mr_results.append((doc_pairs, doc_pairs_id, doc_pairs_info))
#                     ds_mr.save_mr_result_csv(all_pairs=doc_pairs,
#                                              ds_name=name,
#                                              threshold=threshold,
#                                              executors=workers,
#                                              samples_dir=samples_dir,
#                                              heuristic=False)
#                     print('----Done----')
#                 print('\n')
#             mr_results_th[threshold] = mr_results
#         all_mr_results[name] = mr_results_th
#
#     with open(mr_results_path, "wb") as f:
#         pickle.dump(all_mr_results, f)
# else:
#     with open(mr_results_path, "rb") as f:
#         all_mr_results = pickle.load(f)

  0%|          | 0/3 [00:00<?, ?it/s]


----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 4----


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/31 00:32:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2023-05-31 00:32:15.260 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext
2023-05-31 00:35:38.358 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----

----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 7----


2023-05-31 00:38:47.492 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


...

----Done----

----working on random_docs, threshold: 0.9----

----workers: 10, slices: 7----


2023-05-31 02:01:04.010 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----

----working on random_docs, threshold: 0.9----

----workers: 10, slices: 10----


100%|██████████| 3/3 [1:29:27<00:00, 1789.27s/it]                               

----Done----




In [38]:
all_mr_results

{'longest_docs': {0.3: [([(14, 41, 0.3174579152368675),
     (13, 52, 0.32738881252597424),
     (35, 202, 0.35424109042553387),
     (52, 79, 0.30112183296306366),
     (67, 122, 0.318692450693291),
     (172, 183, 0.3285263486427356),
     (86, 129, 0.3712992378735484),
     (118, 161, 0.35285042488420715),
     (122, 317, 0.37970964148307695),
     (189, 356, 0.3268662423291134),
     (126, 313, 0.4016499004283181),
     (217, 232, 0.9773790964474095),
     (64, 67, 0.45003164294711717),
     (197, 204, 0.30997610598713954),
     (384, 387, 0.868626705843517),
     (43, 114, 0.5272453298338095),
     (70, 81, 0.3267552763719171),
     (188, 215, 0.36212884344611407),
     (127, 270, 0.3627020191421163),
     (13, 36, 0.31627673300706594),
     (36, 79, 0.3010163754550505),
     (172, 215, 0.3145808432580045),
     (91, 98, 0.9789265429039566),
     (24, 27, 0.9880591138149927),
     (28, 119, 0.34981578997454754),
     (343, 358, 0.6504553138844349),
     (29, 148, 0.320815494225568

devo estrarre i dati in qualche modo

In [39]:
# print('\nSaving pyspark_results')
# dio = pd.DataFrame.from_dict(
#     dict(zip(range(len(mr_results)), mr_results)),
#     orient='index',
#     columns=[
#         'ds_name',
#         'elapsed',
#         'threshold',
#         'unique_pairs_sim_docs',
#         'workers',
#         'slice_factor'
#     ],
# )
# dio.to_csv('./results/pyspark_results.csv', index=False)
# print( 'Done')

In [40]:
# dio

## Parallel APDS with Heuristic

In [41]:
mr_results_dir = os.path.join(samples_dir, "mr_result")
if not os.path.exists(mr_results_dir):
    os.mkdir(mr_results_dir)

mr_results_path_h = os.path.join(mr_results_dir, "mr_results_h.pkl")

In [42]:
# for collection_name, collection_docs, _ in samples:
all_mr_results_h = ds_mr.exec_mr_apds(tfidf_docs=tfidf_docs,
                                    thresholds=thresholds,
                                    n_executors=n_executors,
                                    n_slices=n_slices,
                                    mr_results_path=mr_results_path_h,
                                    samples_dir=samples_dir,
                                    heuristic=True)

  0%|          | 0/3 [00:00<?, ?it/s]2023-05-31 02:01:37.369 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext



----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 4----


2023-05-31 02:04:00.717 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----

----working on longest_docs, threshold: 0.3----

----workers: 4, slices: 7----


2023-05-31 02:06:29.863 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


...

----Done----

----working on random_docs, threshold: 0.9----

----workers: 10, slices: 7----


2023-05-31 03:30:07.524 | DEBUG    | src.apds.docsim_mapreduce_spark:create_spark_session:262 - Adding /Users/a/GitHub/All-Pairs-Similarity/src.zip to SparkContext


----Done----

----working on random_docs, threshold: 0.9----

----workers: 10, slices: 10----


100%|██████████| 3/3 [1:29:02<00:00, 1780.85s/it]                               

----Done----




In [43]:
all_mr_results_h

{'longest_docs': {0.3: [([(14, 41, 0.3174579152368675),
     (13, 52, 0.32738881252597424),
     (35, 202, 0.35424109042553387),
     (52, 79, 0.30112183296306366),
     (67, 122, 0.318692450693291),
     (172, 183, 0.3285263486427356),
     (86, 129, 0.3712992378735484),
     (118, 161, 0.35285042488420715),
     (122, 317, 0.37970964148307695),
     (189, 356, 0.3268662423291134),
     (126, 313, 0.4016499004283181),
     (217, 232, 0.9773790964474095),
     (64, 67, 0.45003164294711717),
     (197, 204, 0.30997610598713954),
     (384, 387, 0.868626705843517),
     (43, 114, 0.5272453298338095),
     (70, 81, 0.3267552763719171),
     (188, 215, 0.36212884344611407),
     (127, 270, 0.3627020191421163),
     (13, 36, 0.31627673300706594),
     (36, 79, 0.3010163754550505),
     (172, 215, 0.3145808432580045),
     (91, 98, 0.9789265429039566),
     (24, 27, 0.9880591138149927),
     (28, 119, 0.34981578997454754),
     (343, 358, 0.6504553138844349),
     (29, 148, 0.320815494225568

# TODO
- sistemare notebook
- see proportion of time in execution to simulate 1000 doc (run sequential with 1000, parallel 400 then proportionate)
- plot risultati
- finire report (parallelo, risultati)